In [1]:
from imports import *
from pipeline import Pipeline, PipeConfig

In [10]:
with open('devrev_tools.json') as f:
    tools = json.load(f)

with open('devrev_data.json') as f:
    data = json.load(f)

In [3]:
examples = [
    {
        "query": "Allocate the work items I own that are at the 'QA' stage and with 'medium' severity to the current sprint, after prioritizing and summarizing them.",
        "solution": [
            {
                "tool_name": "who_am_i",
                "arguments": []
            },
            {
                "tool_name": "works_list",
                "arguments": [
                    {
                        "argument_name": "owned_by",
                        "argument_value": ["$$PREV[0]"]
                    },
                    {
                        "argument_name": "stage.name",
                        "argument_value": ["QA"]
                    },
                    {
                        "argument_name": "type",
                        "argument_value": ["ticket"]
                    },
                    {
                        "argument_name": "ticket.severity",
                        "argument_value": ["medium"]
                    }
                ]
            },
            {
                "tool_name": "prioritize_objects",
                "arguments": [
                    {
                        "argument_name": "objects",
                        "argument_value": "$$PREV[2]"
                    }
                ]
            },
            {
                "tool_name": "summarize_objects",
                "arguments": [
                    {
                        "argument_name": "objects",
                        "argument_value": "$$PREV[3]"
                    }
                ]
            },
            {
                "tool_name": "get_sprint_id",
                "arguments": []
            },
            {
                "tool_name": "add_work_items_to_sprint",
                "arguments": [
                    {
                        "argument_name": "work_ids",
                        "argument_value": "$$PREV[1]"
                    },
                    {
                        "argument_name": "sprint_id",
                        "argument_value": "$$PREV[2]"
                    }
                ]
            }
        ]
    },

    {
        "query": "Create actionable tasks from the summary of all 'high' severity tickets associated with 'REV-456', get similar work items to those tasks, prioritize by severity, and filter out those that don't need immediate action.",
        "solution": [
            {
                "tool_name": "works_list",
                "arguments": [
                    {
                        "argument_name": "ticket.severity",
                        "argument_value": ["high"]
                    },
                    {
                        "argument_name": "ticket.rev_org",
                        "argument_value": ["REV-456"]
                    },
                    {
                        "argument_name": "type",
                        "argument_value": ["ticket"]
                    },
                    {
                        "argument_name": "ticket.needs_response",
                        "argument_value": True
                    }
                ]
            },
            {
                "tool_name": "summarize_objects",
                "arguments": [
                    {
                        "argument_name": "objects",
                        "argument_value": "$$PREV[0]"
                    }
                ]
            },
            {
                "tool_name": "create_actionable_tasks_from_text",
                "arguments": [
                    {
                        "argument_name": "text",
                        "argument_value": "$$PREV[1]"
                    }
                ]
            },
            {
                "tool_name": "get_similar_work_items",
                "arguments": [
                    {
                        "argument_name": "work_id",
                        "argument_value": "$$PREV[2]"
                    }
                ]
            },
            {
                "tool_name": "prioritize_objects",
                "arguments": [
                    {
                        "argument_name": "objects",
                        "argument_value": "$$PREV[3]"
                    }
                ]
            }
        ]
    }
]

# Pipeline-1: GPT-3.5

In [4]:
pipe = Pipeline(PipeConfig)

In [5]:
import litellm
litellm.set_verbose=False

In [11]:
tools_2 = [
    {
        "tool_name": "assign_work_items",
        "tool_description": "Assigns specified work items to a user",
        "args": [
            {
                "arg_name": "work_item_ids",
                "arg_type": "str",
                "is_array": True,
                "is_required": True,
                "arg_description": "List of work item IDs to be assigned",
                "example": [
                    "TASK-456",
                    "ISSUE-789"
                ]
            },
            {
                "arg_name": "user_id",
                "arg_type": "str",
                "is_array": False,
                "is_required": True,
                "arg_description": "The ID of the user to whom the work items are to be assigned",
                "example": "DEVU-456"
            }
        ],
        "output": {
            "description": "Returns True if the work items were successfully assigned to the user; otherwise, False.",
            "arg_type": "boolean",
            "is_array": False,
            "is_required": True,
            "example": True
        }
    },
    {
        "tool_name": "validate_work_dependency",
        "tool_description": "Checks if specified work items have dependencies and validates their status",
        "args": [
            {
                "arg_name": "work_item_ids",
                "arg_type": "str",
                "is_array": True,
                "is_required": True,
                "arg_description": "List of work item IDs to check for dependencies",
                "example": [
                    "FEAT-456",
                    "BUG-123"
                ]
            }
        ],
        "output": {
            "description": "Returns a list of booleans indicating whether each work item has dependencies and if they are valid.",
            "arg_type": "boolean",
            "is_array": True,
            "is_required": True,
            "example": [
                True,
                False
            ]
        }
    },
    {
        "tool_name": "sync_work_items_with_calendar",
        "tool_description": "Synchronizes work items with a user's calendar to manage deadlines and reminders",
        "args": [
            {
                "arg_name": "user_id",
                "arg_type": "str",
                "is_array": False,
                "is_required": True,
                "arg_description": "The ID of the user whose calendar is to be synchronized",
                "example": "DEVU-567"
            },
            {
                "arg_name": "work_item_ids",
                "arg_type": "str",
                "is_array": True,
                "is_required": True,
                "arg_description": "List of work item IDs to synchronize with the calendar",
                "example": [
                    "FEAT-789",
                    "TASK-890"
                ]
            }
        ],
        "output": {
            "description": "Returns True if the synchronizing process was successful; otherwise, False.",
            "arg_type": "boolean",
            "is_array": False,
            "is_required": True,
            "example": True
        }
    },
    {
        "tool_name": "list_sprint_work_items",
        "tool_description": "Lists all work items associated with a particular sprint.",
        "args": [
            {
                "arg_name": "sprint_id",
                "arg_type": "str",
                "is_array": False,
                "is_required": True,
                "arg_description": "The ID of the sprint to retrieve work items for.",
                "example": "SPR-123"
            }
        ],
        "output": {
            "description": "Returns a list of work items associated with the specified sprint.",
            "arg_type": "str",
            "is_array": True,
            "is_required": True,
            "example": [
                "TASK-001",
                "TASK-002",
                "BUG-101"
            ]
        }
    },
    {
        "tool_name": "filter_by_status",
        "tool_description": "Filters and returns work items based on their status.",
        "args": [
            {
                "arg_name": "status",
                "arg_type": "str",
                "is_array": True,
                "is_required": True,
                "arg_description": "The status(es) to filter work items by.",
                "example": [
                    "In Progress",
                    "To Do",
                    "Open",
                    "Blocked"
                ]
            },
            {
                "arg_name": "work_ids",
                "arg_type": "str",
                "is_array": True,
                "is_required": True,
                "arg_description": "List of work item IDs to be filtered",
                "example": [
                    "TASK-456",
                    "ISSUE-789"
                ]
            }
        ],
        "output": {
            "description": "Returns a list of work items that match the specified status(es).",
            "arg_type": "str",
            "is_array": True,
            "is_required": True,
            "example": [
                "TASK-456",
                "ISSUE-789"
            ]
        }
    },
    {
        "tool_name": "merge_work_items",
        "tool_description": "Combines multiple work items into a single item.",
        "args": [
            {
                "arg_name": "source_work_ids",
                "arg_type": "str",
                "is_array": True,
                "is_required": True,
                "arg_description": "The IDs of the source work items to merge.",
                "example": [
                    "TASK-123",
                    "ISSUE-456"
                ]
            },
            {
                "arg_name": "target_work_id",
                "arg_type": "str",
                "is_array": False,
                "is_required": True,
                "arg_description": "The ID of the target work item to merge into.",
                "example": "TASK-789"
            }
        ],
        "output": {
            "arg_type": "str",
            "is_array": True,
            "is_required": True,
            "description": "Returns a list of work items that were merged into the target work item.",
            "example": [
                "TASK-123",
                "ISSUE-456"
            ]
        }
    }
]

In [21]:
query = """Get all the work items created by ["DEVU-323"] and ["DEVU-432"]. For the items published on media channel, return 10 items prioritized and summarized."""
output = pipe(query,tools,examples)

In [22]:
output

{'embed_calls': 1,
 'return_tools': [],
 'solution': {'action': 'get_work_items',
  'filters': {'created_by': ['DEVU-323', 'DEVU-432'],
   'published_on_media_channel': True},
  'max_results': 10,
  'fields': ['priority', 'summary']},
 'time_taken': 1.3202183246612549,
 'openai_model': 'ft:gpt-3.5-turbo-1106:interiit:jsn2jsn-ak-11-12:8UXjCkwg',
 'embedding_model': 'text-embedding-ada-002',
 'num_embedding_tokens_query': 37,
 'num_embedding_tokens_tool': 0,
 'num_output_tokens': 62,
 'num_input_tokens': 161,
 'embedding_cost_query': 3.7e-06,
 'embedding_cost_tool': 0.0,
 'gpt_input_cost': 0.0014489999999999998,
 'gpt_output_cost': 0.0011159999999999998,
 'total_inference_cost': 0.0025686999999999997,
 'tool_setup_cost': 0.0}